In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [5]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [8]:
def scrape_links(page_number):
    url=f"https://www.jabarnews.com/rubrik/sajabar/page/{page_number}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3',{"class": "entry-title td-module-title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [9]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 25 links from page 1
Total Links: 25


In [10]:

def scrape_url(url):
    response = requests.get(url,headers={'User-Agent': 'Chrome/117.0.0.0'})
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        elem = soup.find('header',{"class": "td-post-title"})
        title_elem = elem.find('h1')
        if title_elem:
            title_text = title_elem.text
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = elem.find('div', {"class": "td-post-author-name"})
        if author_elem:
            author_text = author_elem.find('a')
            author_text = author_text.text
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('time', {"class": "entry-date updated td-module-date"})
        if date_elem:
            date_text = date_elem.text
        else:
            date_text = "Date not found"
            
        #     # Category berita
        # category_elements = soup.find_all('a', {"class": "breadcrumb__link"})
        # if category_elements:
        #     category_text = category_elements[1].text
        # else:
        category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('div', {"class": "td-post-content tagdiv-type"}) 
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"

        results.append({'title': title_text,
                        'author' : author_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'region' : 'jabar',
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [11]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [12]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 25


,title,author,category,date,content,region,link
0,Hati-hati Hujan Lebat Disertai Kilat dan Angin...,Fitri,Category not found,"Kamis, 30 Maret 2023 - 17:11:51",JABARNEWS | BANDUNG – BMKG (Badan Meteorologi ...,jabar,https://www.jabarnews.com/sajabar/hati-hati-hu...
1,21 Kabupaten dan Kota di Jawa Barat Ini Berpot...,Fitri,Category not found,"Kamis, 30 Maret 2023 - 10:15:38",JABARNEWS | BANDUNG – Badan Meteorologi Klimat...,jabar,https://www.jabarnews.com/sajabar/21-kabupaten...
2,Prakiraan Cuaca di Bandung Raya Rabu 29 Maret ...,Fitri,Category not found,"Rabu, 29 Maret 2023 - 08:34:12",JABARNEWS | BANDUNG – Badan Meteorologi Klimat...,jabar,https://www.jabarnews.com/sajabar/prakiraan-cu...
3,Potensi Hujan Disertai Kilat dan Angin Kencang...,Fitri,Category not found,"Selasa, 28 Maret 2023 - 11:59:11",JABARNEWS | BANDUNG – Badan Meteorologi Klimat...,jabar,https://www.jabarnews.com/sajabar/potensi-huja...
4,Informasi Peringatan Dini Tiga Harian untuk Wi...,Fitri,Category not found,"Rabu, 29 Maret 2023 - 13:00:00",JABARNEWS | BANDUNG – Peringatan dini tiga har...,jabar,https://www.jabarnews.com/sajabar/informasi-pe...
5,"Jangan Anggap Biasa Sakit Pinggang, Bisa Jadi ...",Din Saripuddin,Category not found,"Rabu, 27 September 2023 - 10:55:09",JABARNEWS │ BANDUNG – Sakit pinggang merupakan...,jabar,https://www.jabarnews.com/ragam/jangan-anggap-...
6,BMKG: Peringatan Dini Tiga Harian untuk Wilaya...,Fitri,Category not found,"Selasa, 28 Maret 2023 - 09:59:13",JABARNEWS | BANDUNG – Peringatan dini tiga har...,jabar,https://www.jabarnews.com/sajabar/bmkg-peringa...
7,"Daftar Usia Ketum Parpol; Kaesang Termuda, Meg...",Din Saripuddin,Category not found,"Rabu, 27 September 2023 - 09:24:38",JABARNEWS │ BANDUNG – Kaesang Pangarep resmi m...,jabar,https://www.jabarnews.com/ragam/daftar-usia-ke...
8,Prakiraan Cuaca di Jawa Barat Berlaku Kamis 30...,Fitri,Category not found,"Kamis, 30 Maret 2023 - 09:15:27",JABARNEWS | BANDUNG – Prakiraan cuaca di Jawa ...,jabar,https://www.jabarnews.com/sajabar/prakiraan-cu...
9,"Dilarang Tidur dekat Ponsel, Radiasi Smartphon...",Din Saripuddin,Category not found,"Rabu, 27 September 2023 - 11:27:33",JABARNEWS │ BANDUNG – Ponsel pintar atau smart...,jabar,https://www.jabarnews.com/ragam/dilarang-tidur...


In [ ]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/jabarnews_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')